In [53]:
def read(x):
    x = 2 - x
    companies = dict()
    tickers = os.listdir(path="./companies")
    for i in tickers:
        companies[i[17:-4]] = pd.read_csv('companies/'+i)
        companies[i[17:-4]] = companies[i[17:-4]][['Дата', 'Изм. %']]
        companies[i[17:-4]]['Изм. %'] = companies[i[17:-4]]['Изм. %'].str.replace('%', '')
        companies[i[17:-4]]['Изм. %'] = companies[i[17:-4]]['Изм. %'].str.replace(',', '.').astype(float)
        companies[i[17:-4]] = companies[i[17:-4]][::-1]
        if x != 0:
            companies[i[17:-4]] = companies[i[17:-4]][:-x]
    return companies


def predict_one(train, tic):
    model = ARIMA(train, order=(5,1,1))
    model_fit = model.fit()
    result = model_fit.get_forecast()
    ci = result.conf_int(alpha = 0.7)
    print('{:<20}{:<25}{:<20}{:<20}{:<20}'.format(tic, '%.2f' % result.predicted_mean[0],
                                            '%.2f' % result.se_mean,
                                           '%.2f to %.2f' % (ci[0,0], ci[0,1]),
                                             '%.2f' % np.std(train)))
    return result.predicted_mean[0]


def predict(companies):
    exp = {}
    print('{:<20}{:<25}{:<20}{:<20}{:<20}'.format('Тикер', 'Ожидаемая доходность', 'Ошибка', 'Интервал', 'Риск'))
    for i in companies:
        train = companies[i]['Изм. %'].values
        exp[i] = predict_one(train, i)
    return exp


def cov_matrix(companies):
    data = []
    for i in companies:
        data.append(companies[i]['Изм. %'].values)
    cov = np.cov(data)
    return cov
    
    
def tobien(m0p, m0, companies, exp):
    cov = cov_matrix(companies)
    ex = []
    for i in companies:
        ex.append(exp[i])
    ex = np.array([i - m0 for i in ex])
    cov_inv = np.linalg.inv(cov)
    gp2 = np.dot(ex, cov_inv)
    gp2 = np.dot(gp2, ex)
    v = ((m0p - m0) / gp2) * cov_inv
    v = np.dot(v, ex)
    vv = {}
    x = 0
    for i in companies:
        vv[i] = round(v[x], 3)
        x += 1
    return vv


def best10(m0, companies, exp):
    for i in exp:
        if len(companies) > 10:
            if exp[i] < m0:
                companies.pop(i)
        else:
            return companies    
    while len(companies) > 10:
        cov = cov_matrix(companies)
        x = -1
        max = -100000000
        y = 0
        for i in cov:
            i = np.sum(i)
            if max < i:
                max = i
                x = y
            y += 1        
        for i in companies:
            if x == 0:
                companies.pop(i)
                break
            x -= 1          
    return companies


def run(m0, m0p, companies, exp):
    companies = best10(m0, companies, exp)
    v = tobien(m0p, m0, companies, exp)
    sum = 0
    result = 0
    for i in v:
        print('{:>5}: {}'.format(i, v[i].round(3)))
        result += v[i] * exp[i]
        sum += v[i]
    v['unrisk'] = (1 - sum).round(3)
    print('Безрисковый:', (1 - sum).round(3))
    print('\nm0p: ', m0p, '%', sep='')
    return v

def real(companies, v, m0, x):
    x = 2 - x
    result = 0
    for i in v:
        if i == 'unrisk':
            result += v[i] * m0
        else:
            result += companies[i][-1:]['Изм. %'][x] * v[i]
    return result

In [54]:
import pandas as pd
import os
from statsmodels.tsa.arima.model import ARIMA
import numpy as np


m0 = 2.306 / 365 * 7
m0p = round(3 * m0, 3)
print('m0: ', round(m0, 3), '%', sep='')

m0: 0.044%


In [55]:
companies = read(0)
exp = predict(companies)

Тикер               Ожидаемая доходность     Ошибка              Интервал            Риск                
DIS                 1.74                     4.49                0.01 to 3.47        4.42                
MRK                 -0.63                    2.99                -1.79 to 0.52       3.06                
PG                  -1.80                    2.53                -2.77 to -0.83      2.55                
WBA                 2.07                     5.72                -0.13 to 4.28       5.86                
MSFT                2.25                     3.27                0.99 to 3.51        3.66                
DOW                 -0.60                    4.61                -2.38 to 1.17       4.95                
CAT                 -3.42                    3.62                -4.81 to -2.02      4.22                
GS                  -1.77                    4.12                -3.36 to -0.18      4.66                
BA                  0.38                     8

In [56]:
v = run(m0, m0p, companies, exp)

  DIS: 0.002
 MSFT: 0.027
 CSCO: 0.003
 INTC: -0.001
  MMM: -0.001
  JPM: 0.008
  JNJ: 0.027
 AMGN: -0.016
    V: -0.015
  WMT: -0.005
Безрисковый: 0.971

m0p: 0.133%


In [57]:
companies = read(1)
print('Реальная доходность: ', round(real(companies, v, m0, 1), 3), '%', sep='')

Реальная доходность: 0.098%


In [58]:
exp = predict(companies)

Тикер               Ожидаемая доходность     Ошибка              Интервал            Риск                
DIS                 1.38                     4.45                -0.33 to 3.10       4.38                
MRK                 -0.71                    2.98                -1.86 to 0.44       3.05                
PG                  -1.25                    2.55                -2.24 to -0.27      2.53                
WBA                 0.40                     5.84                -1.85 to 2.65       6.06                
MSFT                2.51                     3.23                1.26 to 3.75        3.62                
DOW                 2.06                     4.58                0.30 to 3.83        4.90                
CAT                 -0.26                    3.75                -1.70 to 1.19       4.19                
GS                  2.91                     4.25                1.28 to 4.55        4.67                
BA                  2.64                     9

In [59]:
v = run(m0, m0p, companies, exp)

 MSFT: 0.032
  MCD: -0.005
  CVX: 0.008
   KO: -0.002
  JNJ: 0.009
 AAPL: -0.011
 AMGN: -0.008
    V: -0.012
NKE-2: 0.007
  WMT: -0.008
Безрисковый: 0.99

m0p: 0.133%


In [60]:
companies = read(2)
print('Реальная доходность: ', round(real(companies, v, m0, 2), 3), '%', sep='')

Реальная доходность: -0.063%


In [61]:
exp = predict(companies)

Тикер               Ожидаемая доходность     Ошибка              Интервал            Риск                
DIS                 1.48                     4.45                -0.24 to 3.19       4.38                
MRK                 -0.97                    3.02                -2.14 to 0.19       3.07                
PG                  -0.84                    2.53                -1.82 to 0.13       2.50                
WBA                 0.21                     5.78                -2.02 to 2.43       6.01                
MSFT                2.62                     3.27                1.36 to 3.88        3.61                
DOW                 -0.60                    4.54                -2.35 to 1.15       4.86                
CAT                 -0.37                    3.71                -1.80 to 1.07       4.17                
GS                  1.34                     4.24                -0.30 to 2.97       4.64                
BA                  3.68                     9

In [62]:
v = run(m0, m0p, companies, exp)

  DIS: 0.001
  WBA: -0.004
 MSFT: 0.022
  MCD: -0.007
 INTC: 0.003
  TRV: 0.004
  JPM: 0.006
 AAPL: -0.008
    V: -0.014
NKE-2: 0.009
Безрисковый: 0.988

m0p: 0.133%


In [1]:
sudo python3 -m pip install wheel


SyntaxError: invalid syntax (<ipython-input-1-3d20d3dcf043>, line 1)

In [2]:
sudo python3 -m pip install wheel


SyntaxError: invalid syntax (<ipython-input-2-3d20d3dcf043>, line 1)

In [3]:
!sudo python3 -m pi

Password:


In [4]:
!warface2143647


zsh:1: command not found: warface2143647


In [2]:
pip3 install pandas

SyntaxError: invalid syntax (<ipython-input-2-83358da30272>, line 1)

In [5]:
 !python3 -m pip install pandas
    

In [4]:
!pip install pandas


zsh:1: command not found: pip


In [3]:
import pandas

ModuleNotFoundError: No module named 'pandas'

In [4]:
!conda help


zsh:1: command not found: conda
